In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm 
import pingouin as pg

# Plotting the results from 10 fold CV to use in thesis

### Data frames of all of the scalar values

In [ ]:
dice_scores = pd.DataFrame({
    "UNet" : [0.910550057888031, 0.8982981443405151, 0.9273608922958374, 0.9014230966567993, 0.9083096385002136, 0.9069598913192749, 0.9241437911987305, 0.9256973266601562, 0.9006452560424805, 0.9294912219047546],
    "UNet++" : [0.9157677888870239, 0.891202449798584, 0.9316964149475098, 0.9152337312698364, 0.8811382055282593, 0.921389102935791, 0.9252721667289734, 0.9236195683479309, 0.9285176992416382, 0.9314782023429871],
    "ResUNet" : [0.9152092933654785, 0.9003211259841919, 0.9218366742134094, 0.9018304944038391, 0.8820966482162476, 0.9200831651687622, 0.925200343132019, 0.9211742281913757, 0.9214398264884949, 0.9340419769287109],
    "AttentionUNet" : [0.9240819215774536, 0.8484458923339844, 0.9212237596511841, 0.8991912603378296, 0.8950201869010925, 0.8943902850151062, 0.9190737009048462, 0.9164016842842102, 0.9210432171821594, 0.9349797368049622],
    "UNETR" : [0.8987330198287964, 0.869611382484436, 0.9065892100334167, 0.8820069432258606, 0.8940620422363281, 0.8847061395645142, 0.8995394706726074, 0.9117307662963867, 0.9257276654243469, 0.9259014129638672],
    "Swin-UNETR-v2" : [0.9113326072692871, 0.9012224078178406, 0.9307351112365723, 0.9155080914497375, 0.8978092074394226, 0.9157394170761108, 0.9257823824882507, 0.9179836511611938, 0.9264646768569946, 0.9364802241325378],

})

iou = pd.DataFrame({
    "UNet" :[0.8417069315910339, 0.8240699768066406, 0.867346465587616, 0.8276573419570923, 0.8403932452201843, 0.8402317762374878, 0.8617351651191711, 0.8644774556159973, 0.8286441564559937, 0.8699434399604797],
    "UNet++" : [0.849579393863678, 0.8129923939704895, 0.8737983703613281, 0.8475649952888489, 0.8022400736808777, 0.8561960458755493, 0.8647365570068359, 0.8613064885139465, 0.8728660345077515, 0.8730465173721313],
    "ResUNet" : [0.8503211736679077, 0.8297848701477051, 0.8585097789764404, 0.8286630511283875, 0.8109999895095825, 0.8561197519302368, 0.8646185994148254, 0.8569480180740356, 0.8609398007392883, 0.8776741027832031],
    "AttentionUNet" : [0.8606138825416565, 0.7553737163543701, 0.860005259513855, 0.8227434158325195, 0.8211956024169922, 0.8299015760421753, 0.854082465171814, 0.8505859375, 0.8591924905776978, 0.8788324594497681],
    "UNETR" : [0.8225361704826355, 0.7854092717170715, 0.834078311920166, 0.7979124784469604, 0.8226059079170227, 0.8106845617294312, 0.8252266049385071, 0.841982364654541, 0.8671841621398926, 0.863693118095398],
    "Swin-UNETR-v2" : [0.8424063324928284, 0.8298884034156799, 0.8719298243522644, 0.8488877415657043, 0.8334700465202332, 0.8533872365951538, 0.8658294677734375, 0.8524211645126343, 0.8656458854675293, 0.8819599151611328],
})

precision = pd.DataFrame({
    "UNet" : [ 0.9234275221824646, 0.8713346123695374, 0.9330257177352905, 0.8691607117652893, 0.919681966304779, 0.9411259293556213, 0.9075326919555664, 0.9288071990013123, 0.9155774116516113, 0.9023400545120239],
    "UNet++" : [0.930785596370697, 0.8724910020828247, 0.9469018578529358, 0.9004392027854919, 0.9246425032615662, 0.9326476454734802, 0.9133468270301819, 0.9314680099487305, 0.9322913885116577, 0.9154754877090454],
    "ResUNet" : [0.9348689317703247, 0.8764814734458923, 0.9336535930633545, 0.864737868309021, 0.8817629218101501, 0.9208326935768127, 0.9111323952674866, 0.9491060376167297, 0.9225629568099976, 0.9358997941017151],
    "AttentionUNet" : [0.9206283688545227, 0.8416826725006104, 0.9320287108421326, 0.8633103966712952, 0.9222394824028015, 0.9170163869857788, 0.904541552066803, 0.9288304448127747, 0.9323444962501526, 0.9303169250488281],
    "UNETR" : [0.930235743522644, 0.8399699330329895, 0.930418074131012, 0.8497440218925476, 0.9214296936988831, 0.9005781412124634, 0.8964805603027344, 0.9288619756698608, 0.9299560189247131, 0.9248302578926086],
    "Swin-UNETR-v2" : [0.9091700315475464, 0.8876038193702698, 0.9407762885093689, 0.8824752569198608, 0.9231607913970947, 0.9192206859588623, 0.9263820052146912, 0.9485830664634705, 0.9156669974327087, 0.9279844760894775],
})

recall = pd.DataFrame({
    "UNet" : [0.8980598449707031, 0.9436590671539307, 0.9278637766838074, 0.9488924741744995, 0.9032720923423767, 0.9010984301567078, 0.9519503712654114, 0.9248430132865906, 0.898490309715271, 0.9602572917938232],
    "UNet++" : [0.9002341628074646, 0.9353430271148682, 0.920830249786377, 0.9415451288223267, 0.8533017039299011, 0.9125659465789795, 0.9497424364089966, 0.919495701789856, 0.939242422580719, 0.9504709243774414],
    "ResUNet" : [0.8960528373718262, 0.937889814376831, 0.9167559742927551, 0.9542949795722961, 0.8820680379867554, 0.9224367737770081, 0.9513195157051086, 0.8984007239341736, 0.9334999918937683, 0.930438756942749],
    "AttentionUNet": [0.9311757683753967, 0.8880977034568787, 0.9191577434539795, 0.9560777544975281, 0.8793001174926758, 0.8881308436393738, 0.9454842209815979, 0.9046801328659058, 0.9228026270866394, 0.9427980780601501],
    "UNETR" : [ 0.8647348880767822, 0.9275467991828918, 0.8905948996543884, 0.9324689507484436, 0.8804863095283508, 0.8818406462669373, 0.9132583141326904, 0.8961440324783325, 0.9308604598045349, 0.9321388006210327],
    "Swin-UNETR-v2" : [0.9165830612182617, 0.9333679676055908, 0.9251618981361389, 0.9630469679832458, 0.878855288028717, 0.9200658202171326, 0.9373883008956909, 0.8932986855506897, 0.9437807202339172, 0.9443143010139465] 
})

comp_time_iteration = pd.DataFrame({
    "unet" : []
})

In [ ]:
#Add the folds
dice_scores['Fold'] = [f'Fold_{i+1}' for i in range(10)]
iou['Fold'] = [f'Fold_{i+1}' for i in range(10)]
precision['Fold'] = [f'Fold_{i+1}' for i in range(10)]
recall['Fold'] = [f'Fold_{i+1}' for i in range(10)]

In [ ]:
#model to calculate stats from:
model = "UNet"

print("Precision")
print(f"{precision[model].mean():.3f} +- {precision[model].std():.3f}")
print("\n")
print("Recall")
print(f"{recall[model].mean():.3f} +- {recall[model].std():.3f}")
print("\n")
print("IoU")
print(f"{iou[model].mean():.3f} +- {iou[model].std():.3f}")
print("\n")
print("Dice Scores")
print(f"{dice_scores[model].mean():.3f} +- {dice_scores[model].std():.3f}")
print("\n")



## Violine plot

In [ ]:
fig = plt.figure(figsize=(8, 5))
sns.violinplot(data=dice_scores)
plt.title("10-Fold CV Dice Scores")
plt.ylabel("Validation Dice Score")
plt.xlabel("Model")
plt.grid()
plt.show()

## Checking for normally distributed data
Using Shapiro wilk test. p>0.05 indicate normal distribution



In [ ]:
for model in dice_scores.loc[:, dice_scores.columns != "Fold"]:
    result = stats.shapiro(dice_scores[model])
    print(model)
    print(result)


Using QQ plots for visual representation

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i in range(6):
        model = dice_scores.columns[i]
        stats.probplot(dice_scores[model], dist="norm", plot=axes[i])
        axes[i].set_title(f"Q-Q Plot: {model}")
plt.tight_layout()

## RM ANOVA test

In [ ]:
dice_scores_long = dice_scores.melt(id_vars=['Fold'], var_name='Model', value_name='Dice Score')
rm_anova = pg.rm_anova(dice_scores_long, dv='Dice Score', subject='Fold', within=['Model'], correction=True)
print(rm_anova)

## Friedman Test

In [ ]:
res = stats.friedmanchisquare(dice_scores['UNet'], dice_scores['UNet++'], dice_scores['ResUNet'], dice_scores['AttentionUNet'],dice_scores['UNETR'], dice_scores['Swin-UNETR-v2'])
print(res)

## Computational Time

In [ ]:
comp_time_iteration = pd.DataFrame({
    "UNet" : [3.035, 3.036, 3.04, 3.019, 3.01, 3.01, 3.00, 2.98, 3.00],
    "UNet++" : [3.808, 3.819, 3.843, 3.84, 3.81, 3.76, 3.72, 3.70, 3.67],
    "ResUNet" : [2.761, 2.765, 2.774, 2.78, 2.77, 2.78, 2.75, 2.75, 2.74],
    "AttentionUNet" : [3.074, 3.068, 3.072, 3.066, 3.04, 3.05, 3.04, 3.04, 3.05],
    "UNETR" : [6.344, 6.296, 6.24, 6.21, 6.21, 6.21, 6.21, 6.21, 6.21],
    "Swin-UNETR-v2" : [6.52, 6.49, 6.49, 6.49, 6.49, 6.49, 6.49, 6.48, 6.6],
})

model = "Swin-UNETR-v2"
print("Comp time")
print(f"{comp_time_iteration[model].mean():.3f} +- {comp_time_iteration[model].std():.3f}")
print(f"total time {comp_time_iteration[model].sum():.3f}")